# Apache Spark

## Etapa 1

### Instruções

Inicialmente iremos preparar o ambiente, definindo o diretório onde nosso código será desenvolvido. Para este diretório iremos copiar o arquivo nomes_aleatorios.txt.

Após, em nosso script Python, devemos importar as bibliotecas necessárias:

```
from pyspark.sql import SparkSession

from pyspark import SparkContext, SQLContext
```
Aplicando as bibliotecas do Spark, podemos definir a Spark Session e sobre ela definir o Context para habilitar o módulo SQL

```spark = SparkSession \

                .builder \

                .master("local[*]")\

                .appName("Exercicio Intro") \

                .getOrCreate()```

Nesta etapa, adicione código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. Carregue-o para dentro de um dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

### Código

In [165]:
# Importa a biblioteca findspark para iniciar o serviço do Spark na máquina que hospeda o meu notebook
import findspark
findspark.init()
import pyspark

# Importa demais bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

# Aplica as bibliotefcas do spark
spark = SparkSession\
    .builder\
    .master("local[*]")\
    .appName("Exercicio Intro")\
    .getOrCreate()

# Lê o arquivo de texto
df_nomes = spark.read.csv("Files/nomes_aleatorios.txt")

# Exibe 5 linhas do dataframe
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|    David Staten|
|  Heather Rivera|
|     Lester Wood|
|William Sandoval|
| Gwendolyn Hines|
+----------------+
only showing top 5 rows



## Etapa 2

### Instruções

No Python, é possível acessar uma coluna de um objeto dataframe pelo atributo (por exemplo df_nomes.nome) ou por índice (df_nomes['nome']). Enquanto a primeira forma é conveniente para a exploração de dados interativos, você deve usar o formato de índice, pois caso algum nome de coluna não esteja de acordo seu código irá falhar.

Como não informamos no momento da leitura do arquivo, o Spark não identificou o Schema por padrão e definiu todas as colunas como string. Para ver o Schema, use o método df_nomes.printSchema().

Nesta etapa, será necessário adicionar código para renomear a coluna para Nomes, imprimir o esquema e mostrar 10 linhas do dataframe.

### Código

In [166]:
# Renomeia a coluna para Nomes
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
# Exibe o esquema
df_nomes.printSchema()
# Exibe 10 linhas do dataframe
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+------------------+
|             Nomes|
+------------------+
|      David Staten|
|    Heather Rivera|
|       Lester Wood|
|  William Sandoval|
|   Gwendolyn Hines|
|Constance Mcmillan|
|        Billy Diaz|
|      Marcos Rolfe|
|         Cara Cole|
|         John Gant|
+------------------+
only showing top 10 rows



## Etapa 3

### Intruções

Ao dataframe (df_nomes), adicione nova coluna chamada Escolaridade e atribua para cada linha um dos três valores de forma aleatória: Fundamental, Medio ou Superior.

Para esta etapa, evite usar funções de iteração, como por exemplo: for, while, entre outras. Dê preferência aos métodos oferecidos para próprio Spark.

### Código

In [192]:
# Importa a biblioteca functions
from pyspark.sql import functions as f

df_nomes = df_nomes.withColumn(
  # Gera uma coluna com nome Escolaridade com um array contendo os valores declarados
  'Escolaridade',
  f.array(
  f.lit('Fundamental'),
  f.lit('Medio'),
  f.lit('Superior'),
  # Seleciona de forma aleatória os valores declarados para a coluna Escolaridade
  ).getItem((f.rand()*3).cast("int")
  )
)
# Exibe 10 linhas do dataframe
df_nomes.show(10)

+------------------+------------+---------------+
|             Nomes|Escolaridade|           pais|
+------------------+------------+---------------+
|      David Staten|    Superior|       Paraguai|
|    Heather Rivera|       Medio|       Suriname|
|       Lester Wood| Fundamental|        Uruguai|
|  William Sandoval|       Medio|           Peru|
|   Gwendolyn Hines|       Medio|Guiana Francesa|
|Constance Mcmillan| Fundamental|        Bolívia|
|        Billy Diaz|    Superior|          Chile|
|      Marcos Rolfe| Fundamental|       Colômbia|
|         Cara Cole|    Superior|        Bolívia|
|         John Gant|    Superior|        Bolívia|
+------------------+------------+---------------+
only showing top 10 rows



## Etapa 4

### Instruções

Ao dataframe (df_nomes), adicione nova coluna chamada Pais e atribua para cada linha o nome de um dos 13 países da América do Sul, de forma aleatória.

Para esta etapa, evite usar funções de iteração, como por exemplo: for, while, entre outras. Dê preferência aos métodos oferecidos para próprio Spark.

### Código

In [193]:
# Importa as bibliotecas necessárias
import random
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
# Gera um array contendo o nome dos paíes da América do Sul
countries = ['Argentina', 'Bolívia', 'Brasil', 'Chile', 'Colômbia', 'Equador', 'Guiana', 'Guiana Francesa', 'Paraguai', 'Peru', 'Suriname', 'Uruguai', 'Venezuela']
# Gera uma função que seleciona aleatoriamente países da lista anterior
def choose():
    return random.choice(countries)
# Converte a função python collect() em UDF para aplicar no dataframe
convert = udf(choose, StringType())
# Cria uma nova coluna e aplica a função convertida
df_nomes = df_nomes.withColumn(
    'pais', convert())
# Exibe 10 linhas do dataframe
df_nomes.show(10)

+------------------+------------+---------------+
|             Nomes|Escolaridade|           pais|
+------------------+------------+---------------+
|      David Staten|    Superior|         Brasil|
|    Heather Rivera|       Medio|       Suriname|
|       Lester Wood| Fundamental|        Bolívia|
|  William Sandoval|       Medio|       Suriname|
|   Gwendolyn Hines|       Medio|       Paraguai|
|Constance Mcmillan| Fundamental|       Colômbia|
|        Billy Diaz|    Superior|        Uruguai|
|      Marcos Rolfe| Fundamental|         Brasil|
|         Cara Cole|    Superior|Guiana Francesa|
|         John Gant|    Superior|      Argentina|
+------------------+------------+---------------+
only showing top 10 rows



## Etapa 5

### Intruções

Ao dataframe (df_nomes), adicione nova coluna chamada AnoNascimento e atribua para cada linha um valor de ano entre 1945 e 2010, de forma aleatória. 

Para esta etapa, evite usar funções de iteração, como por exemplo: for, while, entre outras. Dê preferência aos métodos oferecidos para próprio Spark.

### Código